In [ ]:
import numpy as np
import tifffile
import os
import glob
from scipy import ndimage
import networkx as nx
from skimage.measure import label, regionprops
# Remove skimage's binary_dilation - we'll use scipy's version instead
from skimage.graph import route_through_array
from google.colab import drive  # Remove if not using Google Colab

# Mount Google Drive (remove if not using Google Colab)
drive.mount('/content/drive')

# Define input and output paths - MODIFY THESE PATHS
nuclei_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Nuclei'
golgi_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Golgi'  # Or use 'Golgi_Adjusted' if that's where your processed files are
membrane_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Membrane_Adjusted'  # Or use 'Membrane_Adjusted' if needed
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Fused'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Parameters
proximity_distance = 3  # Distance in pixels to consider "attached"
membrane_threshold = 0.7  # Threshold to determine "complete" membrane (0-1)
max_nuclei_distance = 3  # Maximum distance between nuclei centroids to consider merging

# Process each set of files
nuclei_files = glob.glob(os.path.join(nuclei_dir, '*_Nuclei_contrast_bg_tophat_mask.tif'))
print(f"Found {len(nuclei_files)} nuclei mask files")

for nuclei_file in nuclei_files:
    # Extract sequence identifier from filename to match corresponding files
    # Example: denoised_0Pa_A1_19dec21_40x_L2RA_FlatA_seq005_Nuclei_contrast_bg_tophat_mask
    file_name = os.path.basename(nuclei_file)
    # Find the base part (everything up to "seq005")
    parts = file_name.split('_')
    seq_index = -1
    for i, part in enumerate(parts):
        if part.startswith('seq'):
            seq_index = i
            break

    if seq_index == -1:
        print(f"Could not parse sequence identifier from {file_name}. Skipping.")
        continue

    # Create base name from all parts up to and including sequence
    base_parts = parts[:seq_index+1]
    base_name = '_'.join(base_parts)

    # Construct file paths for corresponding golgi and membrane files
    golgi_file = os.path.join(golgi_dir, f"{base_name}_Golgi_contrast_bg_tophat_golgi_labels.tif")
    membrane_file = os.path.join(membrane_dir, f"{base_name}_membrane_mask.tif")

    if not os.path.exists(golgi_file) or not os.path.exists(membrane_file):
        print(f"Missing files for {base_name}. Skipping.")
        continue

    print(f"Processing: {base_name}")

    # Load masks
    nuclei_mask = tifffile.imread(nuclei_file)
    golgi_mask = tifffile.imread(golgi_file)
    membrane_mask = tifffile.imread(membrane_file)

    # Ensure all masks have the same shape
    if not (nuclei_mask.shape == golgi_mask.shape == membrane_mask.shape):
        print(f"  Shape mismatch. Skipping.")
        continue

    # Get unique labels (excluding background 0)
    nuclei_labels = np.unique(nuclei_mask)
    nuclei_labels = nuclei_labels[nuclei_labels > 0]

    # Create a graph where nodes are nuclei
    G = nx.Graph()

    # Add all nuclei as nodes
    for n_label in nuclei_labels:
        G.add_node(n_label)

    # Get centroids of all nuclei for distance calculations
    nuclei_props = regionprops(nuclei_mask)
    nuclei_centroids = {prop.label: prop.centroid for prop in nuclei_props}

    # For each pair of nuclei, check the conditions
    print("  Checking conditions 1-3 for all nuclei pairs...")
    for i, n1_label in enumerate(nuclei_labels):
        n1_binary = (nuclei_mask == n1_label)
        n1_dilated = ndimage.binary_dilation(n1_binary, iterations=proximity_distance)

        for n2_label in nuclei_labels[i+1:]:
            n2_binary = (nuclei_mask == n2_label)

            # CONDITION 1: Nuclei are attached with nothing else in radius
            if np.any(n1_dilated & n2_binary):
                # Check if there's membrane in the vicinity
                region_of_interest = ndimage.binary_dilation(n1_binary | n2_binary, iterations=proximity_distance*2)
                if not np.any(membrane_mask & region_of_interest):
                    G.add_edge(n1_label, n2_label)
                    print(f"    Nuclei {n1_label} and {n2_label} merged: Condition 1 - attached with no membrane")
                    continue

            # CONDITION 2 & 3: Check membrane completeness between close nuclei
            # Get the centroids of the two nuclei
            c1 = nuclei_centroids[n1_label]
            c2 = nuclei_centroids[n2_label]

            # Calculate Euclidean distance between centroids
            distance = np.sqrt((c1[0] - c2[0])**2 + (c1[1] - c2[1])**2)

            # Skip if nuclei are too far apart (not worth checking for merging)
            if distance > max_nuclei_distance:
                continue

            # Create a path between the centroids
            # Convert membrane to a cost array (higher values = harder to traverse)
            # We invert the membrane mask so high values correspond to membrane presence
            cost_array = np.ones_like(membrane_mask, dtype=float)
            cost_array[membrane_mask > 0] = 1000  # High cost for passing through membrane

            # Find the shortest path between centroids
            try:
                indices, weight = route_through_array(
                    cost_array,
                    (int(c1[0]), int(c1[1])),
                    (int(c2[0]), int(c2[1])),
                    fully_connected=True
                )

                # Create a path mask to analyze membrane coverage
                path_mask = np.zeros_like(membrane_mask, dtype=bool)
                indices = np.array(indices)
                path_mask[indices[:, 0], indices[:, 1]] = True

                # Calculate percentage of path covered by membrane
                path_length = len(indices)
                membrane_coverage = np.sum(membrane_mask[path_mask]) / path_length

                # Add a distance factor - require higher membrane coverage for nuclei that are far apart
                # This makes it harder to merge distant nuclei
                distance_factor = min(1.0, distance / max_nuclei_distance)
                adjusted_threshold = membrane_threshold * (1 - distance_factor * 0.3)  # Reduce threshold by up to 30% for close nuclei

                # CONDITION 2: Don't merge if complete membrane separation
                if membrane_coverage > adjusted_threshold:
                    print(f"    Nuclei {n1_label} and {n2_label} NOT merged: Condition 2 - complete membrane separation")
                    continue

                # CONDITION 3: Merge if incomplete membrane division AND nuclei are close
                if distance < max_nuclei_distance * 0.5:  # Only merge if nuclei are within 50% of max distance
                    if 0 < membrane_coverage <= adjusted_threshold:
                      G.add_edge(n1_label, n2_label)
                      print(f"    Nuclei {n1_label} and {n2_label} merged: Condition 3 - incomplete membrane")
                      continue
            except Exception as e:
                print(f"    Error processing path between nuclei {n1_label} and {n2_label}: {e}")

    # CONDITION 4: Use Golgi as a bridge
    print("  Checking condition 4 (Golgi bridges)...")

    # Find which nuclei are connected to each Golgi
    golgi_labels = np.unique(golgi_mask)
    golgi_labels = golgi_labels[golgi_labels > 0]

    golgi_to_nuclei = {g_label: [] for g_label in golgi_labels}

    for g_label in golgi_labels:
        g_binary = (golgi_mask == g_label)
        g_dilated = ndimage.binary_dilation(g_binary, iterations=proximity_distance)

        for n_label in nuclei_labels:
            n_binary = (nuclei_mask == n_label)

            if np.any(g_dilated & n_binary):
                golgi_to_nuclei[g_label].append(n_label)

    # Connect nuclei through Golgi bridges, respecting membrane boundaries
    for g_label, connected_nuclei in golgi_to_nuclei.items():
        if len(connected_nuclei) >= 2:
            for i in range(len(connected_nuclei)):
                for j in range(i+1, len(connected_nuclei)):
                    n1 = connected_nuclei[i]
                    n2 = connected_nuclei[j]

                    # Skip if already connected by other conditions
                    if G.has_edge(n1, n2):
                        continue

                    # Check for membrane separation between these nuclei
                    c1 = nuclei_centroids[n1]
                    c2 = nuclei_centroids[n2]

                    # Calculate Euclidean distance between centroids
                    distance = np.sqrt((c1[0] - c2[0])**2 + (c1[1] - c2[1])**2)

                    # Skip if nuclei are too far apart
                    if distance > max_nuclei_distance:
                        continue

                    # Create cost array for path finding (high cost for membrane)
                    cost_array = np.ones_like(membrane_mask, dtype=float)
                    cost_array[membrane_mask > 0] = 1000

                    try:
                        # Find shortest path between centroids
                        indices, weight = route_through_array(
                            cost_array,
                            (int(c1[0]), int(c1[1])),
                            (int(c2[0]), int(c2[1])),
                            fully_connected=True
                        )

                        # Create path mask and check membrane coverage
                        path_mask = np.zeros_like(membrane_mask, dtype=bool)
                        indices = np.array(indices)
                        path_mask[indices[:, 0], indices[:, 1]] = True

                        path_length = len(indices)
                        membrane_coverage = np.sum(membrane_mask[path_mask]) / path_length

                        # Only merge if membrane is incomplete or absent
                        if membrane_coverage <= membrane_threshold:
                            G.add_edge(n1, n2)
                            print(f"    Nuclei {n1} and {n2} merged: Condition 4 - Golgi bridge with incomplete/no membrane")
                        else:
                            print(f"    Nuclei {n1} and {n2} NOT merged: Golgi bridge blocked by complete membrane")
                    except Exception as e:
                        print(f"    Error processing Golgi path between nuclei {n1} and {n2}: {e}")

    # Find connected components (groups of nuclei that should be merged)
    connected_components = list(nx.connected_components(G))

    # Create a new fused mask
    fused_mask = np.zeros_like(nuclei_mask)

    # Process each connected component
    print(f"  Creating fused mask with {len(connected_components)} components...")
    for i, component in enumerate(connected_components, 1):
        # Create a mask for this component
        component_mask = np.zeros_like(nuclei_mask, dtype=bool)

        # Add all nuclei in this component
        for nucleus_label in component:
            component_mask |= (nuclei_mask == nucleus_label)

        # Assign a unique label to this component
        fused_mask[component_mask] = i

    # Save the fused mask
    output_file = os.path.join(output_dir, f"{base_name}_fused.tif")
    tifffile.imwrite(output_file, fused_mask.astype(np.uint32))
    print(f"  Saved fused mask to {output_file} with {len(connected_components)} structures")

print("All processing complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 16 nuclei mask files
Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001
  Checking conditions 1-3 for all nuclei pairs...
  Checking condition 4 (Golgi bridges)...
  Creating fused mask with 325 components...
  Saved fused mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Fused/denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_fused.tif with 325 structures
Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq002
  Checking conditions 1-3 for all nuclei pairs...
  Checking condition 4 (Golgi bridges)...
  Creating fused mask with 256 components...
  Saved fused mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Fused/denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_fused.tif with 256 structures
Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq003
  Checking conditions 1-3

KeyboardInterrupt: 

In [ ]:
import numpy as np
import tifffile
import os
import glob
from scipy import ndimage
import networkx as nx
from skimage.measure import label, regionprops
from skimage.morphology import binary_dilation
from skimage.graph import route_through_array
from google.colab import drive  # Remove if not using Google Colab

# Mount Google Drive (remove if not using Google Colab)
drive.mount('/content/drive')

# Define input and output paths - MODIFY THESE PATHS
nuclei_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Nuclei'
golgi_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Golgi'  # Or use 'Golgi_Adjusted' if that's where your processed files are
membrane_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Membrane'  # Or use 'Membrane_Adjusted' if needed
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Fused'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Parameters
proximity_distance = 5  # Distance in pixels to consider "attached"
membrane_threshold = 0.7  # Threshold to determine "complete" membrane (0-1)

# Process each set of files
nuclei_files = glob.glob(os.path.join(nuclei_dir, '*_Nuclei_contrast_bg_tophat_mask.tif'))
print(f"Found {len(nuclei_files)} nuclei mask files")

for nuclei_file in nuclei_files:
    # Extract sequence identifier from filename to match corresponding files
    # Example: denoised_0Pa_A1_19dec21_40x_L2RA_FlatA_seq005_Nuclei_contrast_bg_tophat_mask
    file_name = os.path.basename(nuclei_file)
    # Find the base part (everything up to "seq005")
    parts = file_name.split('_')
    seq_index = -1
    for i, part in enumerate(parts):
        if part.startswith('seq'):
            seq_index = i
            break

    if seq_index == -1:
        print(f"Could not parse sequence identifier from {file_name}. Skipping.")
        continue

    # Create base name from all parts up to and including sequence
    base_parts = parts[:seq_index+1]
    base_name = '_'.join(base_parts)

    # Construct file paths for corresponding golgi and membrane files
    golgi_file = os.path.join(golgi_dir, f"{base_name}_Golgi_contrast_bg_tophat_golgi_labels.tif")
    membrane_file = os.path.join(membrane_dir, f"{base_name}_membrane_mask.tif")

    if not os.path.exists(golgi_file) or not os.path.exists(membrane_file):
        print(f"Missing files for {base_name}. Skipping.")
        continue

    print(f"Processing: {base_name}")

    # Load masks
    nuclei_mask = tifffile.imread(nuclei_file)
    golgi_mask = tifffile.imread(golgi_file)
    membrane_mask = tifffile.imread(membrane_file)

    # Ensure all masks have the same shape
    if not (nuclei_mask.shape == golgi_mask.shape == membrane_mask.shape):
        print(f"  Shape mismatch. Skipping.")
        continue

    # Get unique labels (excluding background 0)
    nuclei_labels = np.unique(nuclei_mask)
    nuclei_labels = nuclei_labels[nuclei_labels > 0]

    # Create a graph where nodes are nuclei
    G = nx.Graph()

    # Add all nuclei as nodes
    for n_label in nuclei_labels:
        G.add_node(n_label)

    # Get centroids of all nuclei for distance calculations
    nuclei_props = regionprops(nuclei_mask)
    nuclei_centroids = {prop.label: prop.centroid for prop in nuclei_props}

    # For each pair of nuclei, check the conditions
    print("  Checking conditions 1-3 for all nuclei pairs...")
    for i, n1_label in enumerate(nuclei_labels):
        n1_binary = (nuclei_mask == n1_label)
        n1_dilated = binary_dilation(n1_binary, iterations=proximity_distance)

        for n2_label in nuclei_labels[i+1:]:
            n2_binary = (nuclei_mask == n2_label)

            # CONDITION 1: Nuclei are attached with nothing else in radius
            if np.any(n1_dilated & n2_binary):
                # Check if there's membrane in the vicinity
                region_of_interest = binary_dilation(n1_binary | n2_binary, iterations=proximity_distance*2)
                if not np.any(membrane_mask & region_of_interest):
                    G.add_edge(n1_label, n2_label)
                    print(f"    Nuclei {n1_label} and {n2_label} merged: Condition 1 - attached with no membrane")
                    continue

            # CONDITION 2 & 3: Check membrane completeness between close nuclei
            # Get the centroids of the two nuclei
            c1 = nuclei_centroids[n1_label]
            c2 = nuclei_centroids[n2_label]

            # Create a path between the centroids
            # Convert membrane to a cost array (higher values = harder to traverse)
            # We invert the membrane mask so high values correspond to membrane presence
            cost_array = np.ones_like(membrane_mask, dtype=float)
            cost_array[membrane_mask > 0] = 1000  # High cost for passing through membrane

            # Find the shortest path between centroids
            try:
                indices, weight = route_through_array(
                    cost_array,
                    (int(c1[0]), int(c1[1])),
                    (int(c2[0]), int(c2[1])),
                    fully_connected=True
                )

                # Create a path mask to analyze membrane coverage
                path_mask = np.zeros_like(membrane_mask, dtype=bool)
                indices = np.array(indices)
                path_mask[indices[:, 0], indices[:, 1]] = True

                # Calculate percentage of path covered by membrane
                path_length = len(indices)
                membrane_coverage = np.sum(membrane_mask[path_mask]) / path_length

                # CONDITION 2: Don't merge if complete membrane separation
                if membrane_coverage > membrane_threshold:
                    print(f"    Nuclei {n1_label} and {n2_label} NOT merged: Condition 2 - complete membrane separation")
                    continue

                # CONDITION 3: Merge if incomplete membrane division
                if 0 < membrane_coverage <= membrane_threshold:
                    G.add_edge(n1_label, n2_label)
                    print(f"    Nuclei {n1_label} and {n2_label} merged: Condition 3 - incomplete membrane")
                    continue
            except Exception as e:
                print(f"    Error processing path between nuclei {n1_label} and {n2_label}: {e}")

    # CONDITION 4: Use Golgi as a bridge
    print("  Checking condition 4 (Golgi bridges)...")

    # Find which nuclei are connected to each Golgi
    golgi_labels = np.unique(golgi_mask)
    golgi_labels = golgi_labels[golgi_labels > 0]

    golgi_to_nuclei = {g_label: [] for g_label in golgi_labels}

    for g_label in golgi_labels:
        g_binary = (golgi_mask == g_label)
        g_dilated = binary_dilation(g_binary, iterations=proximity_distance)

        for n_label in nuclei_labels:
            n_binary = (nuclei_mask == n_label)

            if np.any(g_dilated & n_binary):
                golgi_to_nuclei[g_label].append(n_label)

    # Connect nuclei through Golgi bridges, respecting membrane boundaries
    for g_label, connected_nuclei in golgi_to_nuclei.items():
        if len(connected_nuclei) >= 2:
            for i in range(len(connected_nuclei)):
                for j in range(i+1, len(connected_nuclei)):
                    n1 = connected_nuclei[i]
                    n2 = connected_nuclei[j]

                    # Skip if already connected by other conditions
                    if G.has_edge(n1, n2):
                        continue

                    # Check for membrane separation between these nuclei
                    c1 = nuclei_centroids[n1]
                    c2 = nuclei_centroids[n2]

                    # Create cost array for path finding (high cost for membrane)
                    cost_array = np.ones_like(membrane_mask, dtype=float)
                    cost_array[membrane_mask > 0] = 1000

                    try:
                        # Find shortest path between centroids
                        indices, weight = route_through_array(
                            cost_array,
                            (int(c1[0]), int(c1[1])),
                            (int(c2[0]), int(c2[1])),
                            fully_connected=True
                        )

                        # Create path mask and check membrane coverage
                        path_mask = np.zeros_like(membrane_mask, dtype=bool)
                        indices = np.array(indices)
                        path_mask[indices[:, 0], indices[:, 1]] = True

                        path_length = len(indices)
                        membrane_coverage = np.sum(membrane_mask[path_mask]) / path_length

                        # Only merge if membrane is incomplete or absent
                        if membrane_coverage <= membrane_threshold:
                            G.add_edge(n1, n2)
                            print(f"    Nuclei {n1} and {n2} merged: Condition 4 - Golgi bridge with incomplete/no membrane")
                        else:
                            print(f"    Nuclei {n1} and {n2} NOT merged: Golgi bridge blocked by complete membrane")
                    except Exception as e:
                        print(f"    Error processing Golgi path between nuclei {n1} and {n2}: {e}")

    # Find connected components (groups of nuclei that should be merged)
    connected_components = list(nx.connected_components(G))

    # Create a new fused mask
    fused_mask = np.zeros_like(nuclei_mask)

    # Process each connected component
    print(f"  Creating fused mask with {len(connected_components)} components...")
    for i, component in enumerate(connected_components, 1):
        # Create a mask for this component
        component_mask = np.zeros_like(nuclei_mask, dtype=bool)

        # Add all nuclei in this component
        for nucleus_label in component:
            component_mask |= (nuclei_mask == nucleus_label)

        # Assign a unique label to this component
        fused_mask[component_mask] = i

    # Save the fused mask
    output_file = os.path.join(output_dir, f"{base_name}_fused.tif")
    tifffile.imwrite(output_file, fused_mask.astype(np.uint32))
    print(f"  Saved fused mask to {output_file} with {len(connected_components)} structures")

print("All processing complete!")

Mounted at /content/drive
Found 16 nuclei mask files
Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001
  Checking conditions 1-3 for all nuclei pairs...


TypeError: binary_dilation() got an unexpected keyword argument 'iterations'

more recent but previous

In [ ]:
import numpy as np
import tifffile
import os
import glob
from scipy import ndimage
import networkx as nx
from skimage.measure import label, regionprops
from skimage.morphology import binary_dilation
from skimage.graph import route_through_array
from google.colab import drive  # Remove if not using Google Colab

# Mount Google Drive (remove if not using Google Colab)
drive.mount('/content/drive')

# Define input and output paths - MODIFY THESE PATHS
nuclei_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Nuclei'
golgi_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Golgi'
membrane_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Membrane'  # Added membrane directory
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Fused'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Parameters
proximity_distance = 5  # Distance in pixels to consider "attached"
membrane_threshold = 0.7  # Threshold to determine "complete" membrane (0-1)

# Process each set of files
nuclei_files = glob.glob(os.path.join(nuclei_dir, '*_mask.tif'))
print(f"Found {len(nuclei_files)} nuclei mask files")

for nuclei_file in nuclei_files:
    # Extract base name to find corresponding files
    base_name = os.path.basename(nuclei_file).replace('_mask.tif', '')
    golgi_file = os.path.join(golgi_dir, f"{base_name}_golgi_labels.tif")
    membrane_file = os.path.join(membrane_dir, f"{base_name}_membrane_mask.tif")

    if not os.path.exists(golgi_file) or not os.path.exists(membrane_file):
        print(f"Missing files for {base_name}. Skipping.")
        continue

    print(f"Processing: {base_name}")

    # Load masks
    nuclei_mask = tifffile.imread(nuclei_file)
    golgi_mask = tifffile.imread(golgi_file)
    membrane_mask = tifffile.imread(membrane_file)

    # Ensure all masks have the same shape
    if not (nuclei_mask.shape == golgi_mask.shape == membrane_mask.shape):
        print(f"  Shape mismatch. Skipping.")
        continue

    # Get unique labels (excluding background 0)
    nuclei_labels = np.unique(nuclei_mask)
    nuclei_labels = nuclei_labels[nuclei_labels > 0]

    # Create a graph where nodes are nuclei
    G = nx.Graph()

    # Add all nuclei as nodes
    for n_label in nuclei_labels:
        G.add_node(n_label)

    # Get centroids of all nuclei for distance calculations
    nuclei_props = regionprops(nuclei_mask)
    nuclei_centroids = {prop.label: prop.centroid for prop in nuclei_props}

    # For each pair of nuclei, check the conditions
    print("  Checking conditions 1-3 for all nuclei pairs...")
    for i, n1_label in enumerate(nuclei_labels):
        n1_binary = (nuclei_mask == n1_label)
        n1_dilated = binary_dilation(n1_binary, iterations=proximity_distance)

        for n2_label in nuclei_labels[i+1:]:
            n2_binary = (nuclei_mask == n2_label)

            # CONDITION 1: Nuclei are attached with nothing else in radius
            if np.any(n1_dilated & n2_binary):
                # Check if there's membrane in the vicinity
                region_of_interest = binary_dilation(n1_binary | n2_binary, iterations=proximity_distance*2)
                if not np.any(membrane_mask & region_of_interest):
                    G.add_edge(n1_label, n2_label)
                    print(f"    Nuclei {n1_label} and {n2_label} merged: Condition 1 - attached with no membrane")
                    continue

            # CONDITION 2 & 3: Check membrane completeness between close nuclei
            # Get the centroids of the two nuclei
            c1 = nuclei_centroids[n1_label]
            c2 = nuclei_centroids[n2_label]

            # Create a path between the centroids
            # Convert membrane to a cost array (higher values = harder to traverse)
            # We invert the membrane mask so high values correspond to membrane presence
            cost_array = np.ones_like(membrane_mask, dtype=float)
            cost_array[membrane_mask > 0] = 1000  # High cost for passing through membrane

            # Find the shortest path between centroids
            try:
                indices, weight = route_through_array(
                    cost_array,
                    (int(c1[0]), int(c1[1])),
                    (int(c2[0]), int(c2[1])),
                    fully_connected=True
                )

                # Create a path mask to analyze membrane coverage
                path_mask = np.zeros_like(membrane_mask, dtype=bool)
                indices = np.array(indices)
                path_mask[indices[:, 0], indices[:, 1]] = True

                # Calculate percentage of path covered by membrane
                path_length = len(indices)
                membrane_coverage = np.sum(membrane_mask[path_mask]) / path_length

                # CONDITION 2: Don't merge if complete membrane separation
                if membrane_coverage > membrane_threshold:
                    print(f"    Nuclei {n1_label} and {n2_label} NOT merged: Condition 2 - complete membrane separation")
                    continue

                # CONDITION 3: Merge if incomplete membrane division
                if 0 < membrane_coverage <= membrane_threshold:
                    G.add_edge(n1_label, n2_label)
                    print(f"    Nuclei {n1_label} and {n2_label} merged: Condition 3 - incomplete membrane")
                    continue
            except Exception as e:
                print(f"    Error processing path between nuclei {n1_label} and {n2_label}: {e}")

    # CONDITION 4: Use Golgi as a bridge
    print("  Checking condition 4 (Golgi bridges)...")

    # Find which nuclei are connected to each Golgi
    golgi_labels = np.unique(golgi_mask)
    golgi_labels = golgi_labels[golgi_labels > 0]

    golgi_to_nuclei = {g_label: [] for g_label in golgi_labels}

    for g_label in golgi_labels:
        g_binary = (golgi_mask == g_label)
        g_dilated = binary_dilation(g_binary, iterations=proximity_distance)

        for n_label in nuclei_labels:
            n_binary = (nuclei_mask == n_label)

            if np.any(g_dilated & n_binary):
                golgi_to_nuclei[g_label].append(n_label)

    # Connect nuclei through Golgi bridges, respecting membrane boundaries
    for g_label, connected_nuclei in golgi_to_nuclei.items():
        if len(connected_nuclei) >= 2:
            for i in range(len(connected_nuclei)):
                for j in range(i+1, len(connected_nuclei)):
                    n1 = connected_nuclei[i]
                    n2 = connected_nuclei[j]

                    # Skip if already connected by other conditions
                    if G.has_edge(n1, n2):
                        continue

                    # Check for membrane separation between these nuclei
                    c1 = nuclei_centroids[n1]
                    c2 = nuclei_centroids[n2]

                    # Create cost array for path finding (high cost for membrane)
                    cost_array = np.ones_like(membrane_mask, dtype=float)
                    cost_array[membrane_mask > 0] = 1000

                    try:
                        # Find shortest path between centroids
                        indices, weight = route_through_array(
                            cost_array,
                            (int(c1[0]), int(c1[1])),
                            (int(c2[0]), int(c2[1])),
                            fully_connected=True
                        )

                        # Create path mask and check membrane coverage
                        path_mask = np.zeros_like(membrane_mask, dtype=bool)
                        indices = np.array(indices)
                        path_mask[indices[:, 0], indices[:, 1]] = True

                        path_length = len(indices)
                        membrane_coverage = np.sum(membrane_mask[path_mask]) / path_length

                        # Only merge if membrane is incomplete or absent
                        if membrane_coverage <= membrane_threshold:
                            G.add_edge(n1, n2)
                            print(f"    Nuclei {n1} and {n2} merged: Condition 4 - Golgi bridge with incomplete/no membrane")
                        else:
                            print(f"    Nuclei {n1} and {n2} NOT merged: Golgi bridge blocked by complete membrane")
                    except Exception as e:
                        print(f"    Error processing Golgi path between nuclei {n1} and {n2}: {e}")

    # Find connected components (groups of nuclei that should be merged)
    connected_components = list(nx.connected_components(G))

    # Create a new fused mask
    fused_mask = np.zeros_like(nuclei_mask)

    # Process each connected component
    print(f"  Creating fused mask with {len(connected_components)} components...")
    for i, component in enumerate(connected_components, 1):
        # Create a mask for this component
        component_mask = np.zeros_like(nuclei_mask, dtype=bool)

        # Add all nuclei in this component
        for nucleus_label in component:
            component_mask |= (nuclei_mask == nucleus_label)

        # Assign a unique label to this component
        fused_mask[component_mask] = i

    # Save the fused mask
    output_file = os.path.join(output_dir, f"{base_name}_fused.tif")
    tifffile.imwrite(output_file, fused_mask.astype(np.uint32))
    print(f"  Saved fused mask to {output_file} with {len(connected_components)} structures")

print("All processing complete!")

previous method

In [ ]:
import numpy as np
import tifffile
import os
import glob
from scipy import ndimage
import networkx as nx
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
nuclei_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Nuclei'
golgi_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Golgi'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Fused'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Parameters
proximity_distance = 5  # Distance in pixels to consider "close" - adjust based on your image scale

# Process each pair of files
nuclei_files = glob.glob(os.path.join(nuclei_dir, '*_mask.tif'))
print(f"Found {len(nuclei_files)} nuclei mask files")

for nuclei_file in nuclei_files:
    # Extract base name to find corresponding Golgi file
    base_name = os.path.basename(nuclei_file).replace('_mask.tif', '')
    golgi_file = os.path.join(golgi_dir, f"{base_name}_golgi_labels.tif")

    if not os.path.exists(golgi_file):
        print(f"No matching Golgi file found for {base_name}. Skipping.")
        continue

    print(f"Processing: {base_name}")

    # Load masks
    nuclei_mask = tifffile.imread(nuclei_file)
    golgi_mask = tifffile.imread(golgi_file)

    # Ensure both masks have the same shape
    if nuclei_mask.shape != golgi_mask.shape:
        print(f"  Shape mismatch: Nuclei {nuclei_mask.shape}, Golgi {golgi_mask.shape}. Skipping.")
        continue

    # Get unique labels (excluding background 0)
    nuclei_labels = np.unique(nuclei_mask)
    nuclei_labels = nuclei_labels[nuclei_labels > 0]

    golgi_labels = np.unique(golgi_mask)
    golgi_labels = golgi_labels[golgi_labels > 0]

    print(f"  Found {len(nuclei_labels)} nuclei and {len(golgi_labels)} Golgi structures")

    # Create a graph where nodes are nuclei
    G = nx.Graph()

    # Add all nuclei as nodes
    for n_label in nuclei_labels:
        G.add_node(n_label)

    # Create maps to track which golgi connects to which nuclei
    golgi_to_nuclei = {g_label: [] for g_label in golgi_labels}

    # Step 1: Find direct connections between nuclei
    print("  Finding direct connections between nuclei...")

    # Dilate each nucleus to check for direct connections
    for i, n1_label in enumerate(nuclei_labels):
        n1_binary = (nuclei_mask == n1_label)
        n1_dilated = ndimage.binary_dilation(n1_binary, iterations=proximity_distance)

        # Check if this dilated nucleus directly touches other nuclei
        for n2_label in nuclei_labels[i+1:]:  # Only check each pair once
            n2_binary = (nuclei_mask == n2_label)

            # If the dilated nucleus overlaps with another nucleus, add an edge
            if np.any(n1_dilated & n2_binary):
                G.add_edge(n1_label, n2_label)

    # Step 2: Find which nuclei are connected to each Golgi
    print("  Finding connections between nuclei and Golgi...")

    # For each Golgi structure, find which nuclei it connects to
    for g_label in golgi_labels:
        g_binary = (golgi_mask == g_label)
        g_dilated = ndimage.binary_dilation(g_binary, iterations=proximity_distance)

        # Check which nuclei this Golgi connects to
        for n_label in nuclei_labels:
            n_binary = (nuclei_mask == n_label)

            # If the dilated Golgi overlaps with this nucleus, record the connection
            if np.any(g_dilated & n_binary):
                golgi_to_nuclei[g_label].append(n_label)

    # Step 3: Add connections between nuclei through Golgi
    print("  Finding connections between nuclei through Golgi...")

    # For each Golgi, check if it connects multiple nuclei
    for g_label, connected_nuclei in golgi_to_nuclei.items():
        # Only process Golgi that connect to multiple nuclei
        if len(connected_nuclei) >= 2:
            # Add edges between all nuclei connected to this Golgi
            for i in range(len(connected_nuclei)):
                for j in range(i+1, len(connected_nuclei)):
                    G.add_edge(connected_nuclei[i], connected_nuclei[j])

    # Step 4: Find connected components in the graph (nuclei that should be fused)
    print("  Finding connected components...")
    connected_components = list(nx.connected_components(G))

    # Create a new fused mask
    fused_mask = np.zeros_like(nuclei_mask)

    # Step 5: Process each connected component
    for i, component in enumerate(connected_components, 1):
        # Create a binary mask for the nuclei in this component
        component_mask = np.zeros_like(nuclei_mask, dtype=bool)

        # Add all nuclei in this component
        for nucleus_label in component:
            component_mask |= (nuclei_mask == nucleus_label)

        # Add all Golgi that connect nuclei in this component
        for g_label, connected_nuclei in golgi_to_nuclei.items():
            # Check if this Golgi connects multiple nuclei from this component
            connected_to_component = [n for n in connected_nuclei if n in component]

            # Only include Golgi that connect multiple nuclei in this component
            if len(connected_to_component) >= 2:
                component_mask |= (golgi_mask == g_label)

        # Add this component to the fused mask
        fused_mask[component_mask] = i

    # Save the fused mask
    output_file = os.path.join(output_dir, f"{base_name}_fused.tif")
    tifffile.imwrite(output_file, fused_mask.astype(np.uint32))
    print(f"  Saved fused mask to {output_file} with {len(connected_components)} structures")

print("All processing complete!")

Mounted at /content/drive
Found 16 nuclei mask files
No matching Golgi file found for denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Nuclei_contrast_bg_tophat. Skipping.
No matching Golgi file found for denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_Nuclei_contrast_bg_tophat. Skipping.
No matching Golgi file found for denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq003_Nuclei_contrast_bg_tophat. Skipping.
No matching Golgi file found for denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq004_Nuclei_contrast_bg_tophat. Skipping.
No matching Golgi file found for denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq005_Nuclei_contrast_bg_tophat. Skipping.
No matching Golgi file found for denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq006_Nuclei_contrast_bg_tophat. Skipping.
No matching Golgi file found for denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq007_Nuclei_contrast_bg_tophat. Skipping.
No matching Golgi file found for denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq008_Nuclei_contrast_bg_tophat. Skipping.
No matching